# imports

In [26]:
'modified by abdul alkurdi; 10/05/2023'

import pandas as pd
#import cudf
import pickle
import json
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import figure
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.signal import correlate
#import cupy as cp
from scipy.io import wavfile
from scipy import signal, stats
#import peakutils, wfdb, pywt
import csv
import os, statistics
from datetime import datetime
#import heartpy as hp
import json
%matplotlib widget 
# import neurokit2 as nk

from syncfcns import *

# inits

In [27]:
################################
#####   force update ?     #####
################################
force_update = False


radwear_path = '/mnt/c/Users/alkurdi/Desktop/Vansh/data/RADWear/'
redcap_path = radwear_path + 'REDCap responses/'

# load all participant meta data
with open(radwear_path + 'all_p_metadata.json', 'rb') as f:
    all_p_metadata = json.load(f)
# load all participant redcap data

with open(redcap_path + 'redcap_df_2nd.pkl', 'rb') as f:
    redcap_df = pickle.load(f)
    

list_of_participants = all_p_metadata['list of participant IDs']
completed_participants = []


# label definitions
calib_dict = {'meditation': 1, 'cpt': 2, 'baseline': 0}
rot_anx_dict = {'calibration': 0, 'LA': 1, 'HA': 2}

p_calib = {}
p_la = {}
p_ha = {}
p_all = {}
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]

# fcn defs

In [28]:
#########################
# p5 HA day 8 and p9 HA day 10 will be done manually.
# redcap is marked as 0 but data is available
#########################

incomplete = [guy for guy in list_of_participants if guy not in completed_participants]
for p in list_of_participants:
    # check if participant pickle file exists
    if Path(radwear_path + 'Participant ' + str(p) + '/p_' + str(p) + '.pkl').is_file():
        # print(radwear_path+'Participant '+str(p)+'/p_'+str(p)+'.pkl')
        print('participant ', p, ' already processed.')
        completed_participants.append(p)
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]
if force_update:
    incomplete = list_of_participants

participant  4  already processed.
participant  7  already processed.
participant  9  already processed.
participant  12  already processed.
participant  14  already processed.
participant  16  already processed.
participant  17  already processed.
participant  18  already processed.
participant  21  already processed.


In [ ]:
redcap_calib_dict = pd.read_pickle(radwear_path+'/REDCap responses/redcap_calib_dict.pkl')
event_type = ['calm_cal_x2', 'secure_cal_x2', 'tense_cal_x2', 
              'regretful_cal_x2', 'ease_cal_x2', 'upset_cal_x2',
              'worrying_cal_x2', 'rested_cal_x2', 'anxious_cal_x2',
              'comfort_cal_x2', 'self_conf_cal_x2', 'nervous_cal_x2',
              'jittery_cal_x2', 'strun_cal_x2', 'relaxed_cal_x2',
              'content_cal_x2', 'worried_cal_x2', 'excited_cal_x2', 
              'joyful_cal_x2', 'pleasant_cal_x2', 'calm_cal_y6', 
              'tense_cal_y6', 'upset_cal_y6', 'relax_cal_y6',
              'content_cal_y6', 'worried_cal_y6', 'calm_cal_y6_post',
              'tense_cal_y6_post', 'upset_cal_y6_post', 'relax_cal_y6_post',
              'content_cal_y6_post', 'worry_cal_y6_post', 'calm_cal_y6_cold',
              'tense_cal_y6_cold', 'upset_cal_y6_cold', 'relax_cal_y6_cold', 
              'content_cal_y6_cold', 'worry_cal_y6_cold', 'baseline_calibration_complete']
tag_info = {4: '',
            7: 'skip 0',
            9: '',
            12: '',
            14: 'add 60 to 2',
            16: 'skip 0',
            17: 'skip 0',
            18: 'skip 01',
            19: 'skip 0',
            21: 'skip 0',
}

p_calib = {}
for p in incomplete:
    print('procesing participant: ', p, ' calibration data...')
    p_all[p] = {}
    p_la[p] = {}
    p_ha[p] = {}
    p_path = radwear_path+'Participant '+str(p)
    # load participant e4 data
    e4sn = all_p_metadata[str(p)]['e4sn']
    calibration_files = all_p_metadata[str(p)]['calibration']
    LA = all_p_metadata[str(p)]['LA']
    HA = all_p_metadata[str(p)]['HA']
    
    # load calibration data
    e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['calibration'][0]
    hx_num = str(all_p_metadata[str(p)]['calibration'][1])
    p_calib[p] = read_sync_return(p_path, e4_num, hx_num)
    print('marker 1')
    #####################
    # calibration data
    #####################
    
    p_calib[p]['rot_label'] = rot_anx_dict['calibration'] * np.ones(len(p_calib[p]['ECG'])) # add label to designate calibration segment
    # get calibration event tags 
    tags = pd.read_csv(
        radwear_path + 'Participant ' + str(p) + '/' + e4_num + '/tags.csv', header=None
    )
    #print(tag_info[p])
    tag_command = tag_info[p].split(' ')
    
    if len(tag_command[-1]) ==1 and tag_command[0] == 'skip':
        tags = tags.drop(int(tag_command[-1]))
    elif len(tag_command[-1]) ==2:
        #print(tag_command[-1][0], tag_command[-1][1])
        tags.drop([int(tag_command[-1][0]),int(tag_command[-1][1])], axis=0 ,inplace=True)
    elif len(tag_command[-1]) ==1 and tag_command[0] == 'add':
        new_row = pd.DataFrame(tags.values[-1]+60,columns=tags.columns)
        tags = pd.concat([tags,new_row], ignore_index=True)
    else:
        tag_command = None
    tags.reset_index(drop=True, inplace=True) 

    print('marker 2')
    # add label to designate calibration segment. same length as bvp to make things easier
    p_calib[p]['calib_label'] = p_calib[p]['BVP'].copy(deep=True) # add label to designate calibration segment
    p_calib[p]['calib_label'].drop(columns=['BVP','Second'], inplace=True)
    p_calib[p]['calib_label']['calib_label'] = 0
    #display(p_calib[p]['calib_label'])
    p_calib[p]['calib_label']['calib_label'][(p_calib[p]['calib_label']['Timestamp'] > tags.iloc[0].values[0]) & \
                                            (p_calib[p]['calib_label']['Timestamp'] < tags.iloc[1].values[0])] \
                                                = calib_dict['meditation']
    p_calib[p]['calib_label']['calib_label'][(p_calib[p]['calib_label']['Timestamp'] > tags.iloc[2].values[0]) & \
                                                (p_calib[p]['calib_label']['Timestamp'] < tags.iloc[3].values[0])] \
                                                    = calib_dict['cpt']
                                                     
    
    
    #####################
    # HA data
    #####################
    if all_p_metadata[str(p)]['complete days'][0] > 0:
        # loop for LA
        for i in range(all_p_metadata[str(p)]['complete days'][0]): 
            if not (all_p_metadata[str(p)]['RedCap available'][0][i] == 0 or all_p_metadata[str(p)]['LA'][0][i] == 0 or all_p_metadata[str(p)]['LA'][1][i] == 0): 
                print('procesing participant: ', p, ' LA day ', i+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['LA'][0][i]
                hx_num = str(all_p_metadata[str(p)]['LA'][1][i])
                
                p_la[p][i] = read_sync_return(p_path, e4_num, hx_num)
                p_la[p][i]['rot_label']= rot_anx_dict['LA'] * np.ones(len(p_la[p][i]['ECG'])) 
            else:
                p_la[p][i] = {} 
    #####################
    # LA data
    #####################
    if all_p_metadata[str(p)]['complete days'][1] > 0:
        # loop for HA    
        for j in range(all_p_metadata[str(p)]['complete days'][1]): # loop for HA
            if not (all_p_metadata[str(p)]['RedCap available'][1][j] == 0 or all_p_metadata[str(p)]['HA'][0][j] == 0 or all_p_metadata[str(p)]['HA'][1][j] == 0): 
                print('procesing participant: ', p, ' HA day ', j+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['HA'][0][j]
                hx_num = str(all_p_metadata[str(p)]['HA'][1][j])
                
                p_ha[p][j] = read_sync_return(p_path, e4_num, hx_num)
                p_ha[p][j]['rot_label']= rot_anx_dict['HA'] * np.ones(len(p_ha[p][j]['ECG']))
            else:
                p_ha[p][j] = {}
    
    print('participant ', p, ' done.')
    if True:
        try:
            p_all[p]['calib']= p_calib[p]
        except:
            print('participant ', p, ' has no calibration data.')
            p_all[p]['calib'] = {}
        try:    
            p_all[p]['LA'] = p_la[p]
        except:
            print('participant ', p, ' has no LA data.')
            p_all[p]['LA'] = {}      
        try: 
            p_all[p]['HA'] = p_ha[p]
        except:
            print('participant ', p, ' has no HA data.')
            p_all[p]['HA'] = {}
        #save this participant data to a pickle file
        
    with open(p_path+'/p_'+str(p)+'.pkl', 'wb') as handle:
            pickle.dump(p_all[p], handle, protocol=pickle.HIGHEST_PROTOCOL)
    #p_all[p].to_parquet('/p_'+str(p)+'.parquet', engine='pyarrow')

    print('participant ', p, ' pickled.')

    #df = pd.DataFrame.from_dict(p_all[p])
    #df.to_parquet(p_path+'/p_'+str(p)+'.parquet')
    #print('participant ', p, ' parqued.')

    #completed_participants.append(p)
    print('-----------------------------------')
    e4sn = None
    calibration_files = None 
    LA = None
    HA = None
    e4_num = None
    hx_num = None
    tags = None
    tag_command = None
print('completed participants: ', set(completed_participants))
# print('incomplete participants: ', incomplete) doesn't update after loop completes them. 

In [78]:
p=16
p_all[p]['calib'].keys()

p_all_like_wesad = {}


p_like_wesad = {}
p_like_wesad['subject'] = p
p_like_wesad['signal'] = {}
p_like_wesad['signal']['chest'] = {}
p_like_wesad['signal']['chest']['ACC'] = p_all[p]['calib']['ACC_hx']
p_like_wesad['signal']['chest']['ECG'] = p_all[p]['calib']['ECG']
p_like_wesad['signal']['chest']['EDA'] = p_all[p]['calib']['EDA']
p_like_wesad['signal']['chest']['Resp'] = p_all[p]['calib']['Resp']
p_like_wesad['signal']['wrist'] = {}
p_like_wesad['signal']['wrist']['ACC'] = p_all[p]['calib']['ACC_e4']
p_like_wesad['signal']['wrist']['BVP'] = p_all[p]['calib']['BVP']
p_like_wesad['signal']['wrist']['EDA'] = p_all[p]['calib']['EDA']
p_like_wesad['signal']['wrist']['TEMP'] = p_all[p]['calib']['TEMP']
p_like_wesad['label'] = {}
p_like_wesad['label']['activity'] = p_all[p]['calib_label']
p_like_wesad['label']['valence'] = p_all[p]['rot_label']



p_all_like_wesad[p] = 

dict_keys(['ECG', 'BVP', 'BR', 'RESP', 'B_PH', 'TEMP', 'EDA', 'HR', 'ACC_hx', 'ACC_e4', 'rot_label', 'calib_label'])

# formatting data

In [ ]:
# all_p = {}
# load each participant's data
force_update = True
for p in list_of_participants:
    p_path = radwear_path + 'Participant ' + str(p)
    p_df = pd.DataFrame()

    # check if file exist
    a = (
        'available.'
        if os.path.isfile(p_path + '/formatted_p_' + str(p) + '.pkl')
        else ' not available.'
    )
    print('pickle file for participant ' + str(p) + ' is ' + a)

    # all_p[p] = p_data ## this takes too much memory so i will just load each p when needed
    if not (os.path.isfile(radwear_path + 'formatted_p_' + str(p) + '.pkl')) or force_update:
        with open(p_path + '/formatted_p_' + str(p) + '.pkl', 'rb') as f:
            p_data = pickle.load(f)
        print('participant ' + str(p) + ' data loaded.')
        print('-----------------------')
        print('Participant: ', p)
        print('-----------------------')
        print('la days: ', len(p_data['HA'].keys()))
        print('ha days: ', len(p_data['LA'].keys()))
        print('-----------------------')
        # e4 contains BVP, EDA, TEMP, ACC, IBI, HR, HRV, tags
        # hx contains ECG, ACC, BR
        p_redcap = redcap_df.loc[
            redcap_df['participant'] == p
        ]  # redcap data for participant p

        ## this is for participant p

        for day in list(p_data['LA'].keys()):
            proceed1 = bool(len(p_data['LA'][day])) and bool(
                all_p_metadata[str(p)]['RedCap available'][0][day]
            )

            if proceed1:
                LA_df = p_data['LA'][day]['ECG'].copy()
                del p_data['LA'][day]['ECG']
                resample_ratio = round(len(LA_df))

                LA_df['BVP'] = signal.resample(
                    p_data['LA'][day]['BVP']['BVP'], resample_ratio
                )
                LA_df['participant'] = p
                del p_data['LA'][day]['ECG']

                LA_df['EDA'] = signal.resample(
                    p_data['LA'][day]['EDA']['EDA'], resample_ratio
                )
                del p_data['LA'][day]['EDA']
                LA_df['TEMP'] = signal.resample(
                    p_data['LA'][day]['TEMP']['Temp'], resample_ratio
                )
                del p_data['LA'][day]['TEMP']
                LA_df['BR'] = signal.resample(
                    p_data['LA'][day]['BR']['breathing_rate'], resample_ratio
                )
                del p_data['LA'][day]['BR']
                LA_df['ACCx_hx'] = signal.resample(
                    p_data['LA'][day]['ACC_hx']['Acceleration_X'], resample_ratio
                )
                del p_data['LA'][day]['ACC_hx']['Acceleration_X']
                LA_df['ACCy_hx'] = signal.resample(
                    p_data['LA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio
                )
                del p_data['LA'][day]['ACC_hx']['Acceleration_Y']
                LA_df['ACCz_hx'] = signal.resample(
                    p_data['LA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio
                )
                del p_data['LA'][day]['ACC_hx']['Acceleration_Z']
                LA_df['ACCx_e4'] = signal.resample(
                    p_data['LA'][day]['ACC_e4']['Acc_X'], resample_ratio
                )
                del p_data['LA'][day]['ACC_e4']['Acc_X']
                LA_df['ACCy_e4'] = signal.resample(
                    p_data['LA'][day]['ACC_e4']['Acc_Y'], resample_ratio
                )
                del p_data['LA'][day]['ACC_e4']['Acc_Y']
                LA_df['ACCz_e4'] = signal.resample(
                    p_data['LA'][day]['ACC_e4']['Acc_Z'], resample_ratio
                )
                del p_data['LA'][day]['ACC_e4']['Acc_Z']
                LA_df['rot_label'] = rot_anx_dict['LA'] * np.ones(resample_ratio)
                LA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio, itemsize=15, unicode=True)
                        A[:] = (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                            column
                        ]
                        LA_df[column] = A
                    else:
                        LA_df[column] = (
                            np.ones(resample_ratio)
                            * (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                                column
                            ]
                        )

                print(
                    'Done with day ',
                    day,
                    'date: ',
                    (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                        'daily_check_in_date'
                    ],
                    ' of LA rotation for participant ',
                    p,
                )
                p_df = pd.concat([p_df, LA_df], ignore_index=True, sort=False)
                # print('display LA_df after LA_df')
                # display(LA_df)

        for day in list(p_data['HA'].keys()):
            proceed2 = bool(len(p_data['HA'][day])) and bool(
                all_p_metadata[str(p)]['RedCap available'][1][day]
            )
            if proceed2:
                HA_df = p_data['HA'][day]['BVP'].copy()
                del p_data['HA'][day]['BVP']
                HA_df['participant'] = p
                resample_ratio = round(len(HA_df))
                HA_df['ECG'] = signal.resample(
                    p_data['HA'][day]['ECG']['ECG'], resample_ratio
                )
                del p_data['HA'][day]['ECG']
                # no need to increase sampling rate of signals. pandas can accept unevenly sampled data
                HA_df['EDA'] = signal.resample(
                    p_data['HA'][day]['EDA']['EDA'], resample_ratio
                )
                del p_data['HA'][day]['EDA']
                HA_df['TEMP'] = signal.resample(
                    p_data['HA'][day]['TEMP']['Temp'], resample_ratio
                )
                del p_data['HA'][day]['TEMP']
                HA_df['BR'] = signal.resample(
                    p_data['HA'][day]['BR']['breathing_rate'], resample_ratio
                )
                del p_data['HA'][day]['BR']
                HA_df['ACCx_hx'] = signal.resample(
                    p_data['HA'][day]['ACC_hx']['Acceleration_X'], resample_ratio
                )
                del p_data['HA'][day]['ACC_hx']['Acceleration_X']
                HA_df['ACCy_hx'] = signal.resample(
                    p_data['HA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio
                )
                del p_data['HA'][day]['ACC_hx']['Acceleration_Y']
                HA_df['ACCz_hx'] = signal.resample(
                    p_data['HA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio
                )
                del p_data['HA'][day]['ACC_hx']['Acceleration_Z']
                HA_df['ACCx_e4'] = signal.resample(
                    p_data['HA'][day]['ACC_e4']['Acc_X'], resample_ratio
                )
                del p_data['HA'][day]['ACC_e4']['Acc_X']
                HA_df['ACCy_e4'] = signal.resample(
                    p_data['HA'][day]['ACC_e4']['Acc_Y'], resample_ratio
                )
                del p_data['HA'][day]['ACC_e4']['Acc_Y']
                HA_df['ACCz_e4'] = signal.resample(
                    p_data['HA'][day]['ACC_e4']['Acc_Z'], resample_ratio
                )
                del p_data['HA'][day]['ACC_e4']['Acc_Z']
                HA_df['rot_label'] = rot_anx_dict['HA'] * np.ones(resample_ratio)
                HA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio, itemsize=15, unicode=True)
                        A[:] = (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                            column
                        ]
                        HA_df[column] = A
                    else:
                        HA_df[column] = (
                            np.ones(resample_ratio)
                            * (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                                column
                            ]
                        )

                print(
                    'Done with day ',
                    day,
                    'date: ',
                    (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][
                        'daily_check_in_date'
                    ],
                    ' of HA rotation for participant ',
                    p,
                )
                p_df = pd.concat([p_df, HA_df], ignore_index=True, sort=False)

        calib_df = p_data['calib']['BVP'].copy()
        del p_data['calib']['BVP']
        resample_ratio = round(len(calib_df))

        calib_df['ECG'] = signal.resample(p_data['calib']['ECG']['ECG'], resample_ratio)
        del p_data['calib']['ECG']
        # calib_df['BVP'] = signal.resample(p_data['calib']['BVP']['BVP'], resample_ratio)
        calib_df['EDA'] = signal.resample(p_data['calib']['EDA']['EDA'], resample_ratio)
        del p_data['calib']['EDA']
        calib_df['TEMP'] = signal.resample(
            p_data['calib']['TEMP']['Temp'], resample_ratio
        )
        del p_data['calib']['TEMP']
        calib_df['BR'] = signal.resample(
            p_data['calib']['BR']['breathing_rate'], resample_ratio
        )
        del p_data['calib']['BR']
        calib_df['ACCx_hx'] = signal.resample(
            p_data['calib']['ACC_hx']['Acceleration_X'], resample_ratio
        )
        del p_data['calib']['ACC_hx']['Acceleration_X']
        calib_df['ACCy_hx'] = signal.resample(
            p_data['calib']['ACC_hx']['Acceleration_Y'], resample_ratio
        )
        del p_data['calib']['ACC_hx']['Acceleration_Y']
        calib_df['ACCz_hx'] = signal.resample(
            p_data['calib']['ACC_hx']['Acceleration_Z'], resample_ratio
        )
        del p_data['calib']['ACC_hx']['Acceleration_Z']
        calib_df['ACCx_e4'] = signal.resample(
            p_data['calib']['ACC_e4']['Acc_X'], resample_ratio
        )
        del p_data['calib']['ACC_e4']['Acc_X']
        calib_df['ACCy_e4'] = signal.resample(
            p_data['calib']['ACC_e4']['Acc_Y'], resample_ratio
        )
        del p_data['calib']['ACC_e4']['Acc_Y']
        calib_df['ACCz_e4'] = signal.resample(
            p_data['calib']['ACC_e4']['Acc_Z'], resample_ratio
        )
        del p_data['calib']['ACC_e4']['Acc_Z']
        calib_df['rot_label'] = rot_anx_dict['calibration'] * np.ones(resample_ratio)
        calib_df['calib_label'] = np.ones(resample_ratio)  # complete later
        redcap_dfcalib = (
            redcap_df  # a cheat for now since i don't have redcap data for calibration
        )
        columns_list = redcap_dfcalib.columns
        ############################################
        # redcap for calibration doesn't exist yet #
        ############################################

        for column in columns_list:
            # print(column)
            # print((redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column]) # this is incorrect for now, replace with calibration data later
            if column == 'daily_check_in_date':
                A = np.chararray(resample_ratio, itemsize=15, unicode=True)
                A[:] = (redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][
                    column
                ]  # this is incorrect for now, replace with calibration data later
                calib_df[column] = A
            else:
                calib_df[column] = (
                    np.ones(resample_ratio)
                    * (redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][
                        column
                    ]
                )  # this is incorrect for now, replace with calibration data later

        p_df = pd.concat([p_df, calib_df], ignore_index=True, sort=False)
        # print('display calib_df after calib')
        # display(calib_df)
        # display(calib_df)

        # save the data
        print('Pickling processed data for participant ' + str(p) + ' ...')
        with open(radwear_path + 'formatted_p_' + str(p) + '.pkl', 'wb') as f:
            pickle.dump(p_df, f)
        print('data pickled...')

    else:
        print('processed data for participant ' + str(p) + ' is already available!')
        # load the data
        with open(radwear_path +'formatted_p_'+str(p)+'.pkl', 'rb') as f:
            p_df = pickle.load(f)

# combining into 1 DF

In [ ]:
all_p_df = pd.DataFrame()

for p in list_of_participants:
    p_path = radwear_path
    # p_df = pd.DataFrame()

    # check if file exist
    a = (
        'available.'
        if os.path.isfile(p_path + '/p_' + str(p) + '.pkl')
        else ' not available.'
    )
    print('processed data for participant ' + str(p) + ' is ' + a)
    print('Loading...')
    with open(p_path + '/p_' + str(p) + '.pkl', 'rb') as f:
        p_df = pickle.load(f)
    print('participant ' + str(p) + ' data loaded.')
    print('-----------------------')
    all_p_df = pd.concat([all_p_df, p_df], ignore_index=True, sort=False)
    display(p_df.sample(3))
    all_p_df.columns
print('pickling all participants file...')
all_p_df.to_pickle(radwear_path + 'all_participants.pkl')
print('pickling completed. ')
print('DF for all participants is saved')